# Validation (for selection), Testing (for evaluation)

*Intuition: Training Set vs. Test Set vs. Validation Set - YouTube*
<br>
*https://www.youtube.com/watch?v=swCf51Z8QDo*

### Data source: 

*common-voice2 - Kaggle*
<br>
*https://www.kaggle.com/datasets/danielgraham1997/commonvoice2*

## Libraries and functions

In [1]:
from ipynb.fs.full.Imports_and_Functions import *

## Local directories and data

In [2]:
TTV = ['train', 'test', 'validation']
main_dir = 'commonvoice'
data_ext = '.tsv'
data_sep = '\t'
aud_ext = '.wav'
clips = 'clips\\'

TR_dir = main_dir + '\\' + TTV[0] + '\\'
TE_dir = main_dir + '\\' + TTV[1] + '\\'
VA_dir = main_dir + '\\' + TTV[2] + '\\'

TR_dir_clips = TR_dir + clips
TE_dir_clips = TE_dir + clips
VA_dir_clips = VA_dir + clips

TR_df = pd.read_csv(main_dir + '\\' + TTV[0] + '\\' + TTV[0] + data_ext, sep= data_sep)
TE_df = pd.read_csv(main_dir + '\\' + TTV[1] + '\\' + TTV[1] + data_ext, sep= data_sep)
VA_df = pd.read_csv(main_dir + '\\' + TTV[2] + '\\' + TTV[2] + data_ext, sep= data_sep)

TTV_dir = [TR_dir, TE_dir, VA_dir]
TTV_dir_clips = [TR_dir_clips, TE_dir_clips, VA_dir_clips]
TTV_df = [TR_df, TE_df, VA_df]


### Localize
temp_dir_clips = TR_dir_clips
temp_df = TR_df
temp_df1 = TE_df
temp_df2 = VA_df
temp_list_df = TTV_df
temp_aud_ext = aud_ext

VA_df.head()

,client_id,path,sentence,up_votes,down_votes,age,gender,accent
0,1514d25318055208d45730bfb6f0391af99036e7778e78...,common_voice_en_102385,In my book of memory,3,0,fifties,female,us
1,cb5bd9ad996218619531511ae2600aa61055005f910e7c...,common_voice_en_1027059,Little things please little minds,2,0,twenties,male,us
2,013f1c38752256d677ba8e23cd5c617a18e22398ed4b40...,common_voice_en_110778,Owls are always intellectual in fairy-tales.,2,1,twenties,female,us
3,013f1c38752256d677ba8e23cd5c617a18e22398ed4b40...,common_voice_en_110779,You should read the new book Mark Twain's writ...,2,1,twenties,female,us
4,cd799a2e46552a3e15e9d08637bde7250909a627017971...,common_voice_en_115744,We all hope I am right.,3,0,thirties,male,NaN


# VALIDATION

('num_onsets',)
<br>
('num_onsets', 'onset_stren_mean')
<br>
('num_onsets', 'onset_stren_mean', 'dom_freq')



In [3]:
TR_MED_df = pd.read_csv(TTV[0][0:2].upper() + '_MED_df' + data_ext, sep=data_sep)
TE_MED_df = pd.read_csv(TTV[1][0:2].upper() + '_MED_df' + data_ext, sep=data_sep)
VA_MED_df = pd.read_csv(TTV[2][0:2].upper() + '_MED_df' + data_ext, sep=data_sep)

features = ['age_num', 'gender', 'accent', 
            'num_onsets', 'onset_stren_mean', 'dom_freq']
feats = features.copy()
feat = ['num_onsets']
two_feat = ['num_onsets', 'onset_stren_mean']
cats = ['age_num', 'gender', 'accent']
quants = ['num_onsets', 'onset_stren_mean', 'dom_freq']
want_pred = 'MEDIAN_func_num'

### 1. Linear Regression

In [4]:
features = feat.copy()
X = TR_MED_df[features].copy()
y = TR_MED_df[want_pred].copy()

percentage = 0.80
l = len(VA_MED_df[want_pred].iloc[:].to_list())
perci = int(l*percentage)

X_val = VA_MED_df[features].copy()
X_val_TR = VA_MED_df[features].copy().iloc[:perci,:]
X_val_VA = VA_MED_df[features].copy().iloc[perci:,:]

y_val = VA_MED_df[want_pred].copy()
y_val_TR = VA_MED_df[want_pred].iloc[:perci]
y_val_VA = VA_MED_df[want_pred].iloc[perci:]

# Local model
model = linear_model.LinearRegression()

# General
model.fit(X,y)
score = model.score(X,y)
pred = model.predict(VA_MED_df[features])
print('Model score:', score)
print("Validation MSE:", 
      mean_squared_error(VA_MED_df[want_pred], pred), '\n'*2)

# BASELINES

# Baseline 1 - msl = 15, st = -2, targ = 15
pred = list(map(CountWordsAudio_apply, VA_MED_df['aud_path']))
for i in range(len(pred)):
    pred[i] = pred[i] / VA_MED_df['num_words'][i]
print('Validation MSE of baseline "msl=15, st=-2, targ=15":', 
      mean_squared_error(y_val, pred), '\n')

# Baseline 2 - Average
pred = [st.mean(y_val)]*l
print('Validation MSE of baseline "Average":', 
      mean_squared_error(y_val, pred), '\n')

# Baseline 3 - Trend
mlr = LinearRegression()
mlr.fit(X_val_TR, y_val_TR)
pred = mlr.predict(X_val_VA)
score = mlr.score(X_val_VA, y_val_VA)
print('Model score of baseline "Trend":', score)
print('Validation MSE of baseline: "Trend"', 
      mean_squared_error(y_val_VA, pred), '\n')

Model score: 0.08258976285484898
Validation MSE: 336.16257390230044 


Validation MSE of baseline "msl=15, st=-2, targ=15": 1512.8667230214417 

Validation MSE of baseline "Average": 374.549375 

Model score of baseline "Trend": -0.06770883999111232
Validation MSE of baseline: "Trend" 282.83190097598947 



### 2. Multiple Linear Regression

### a.  3D

*https://aegis4048.github.io/mutiple_linear_regression_and_visualization_in_python*

In [5]:
features = two_feat.copy()
X = TR_MED_df[features].copy()
y = TR_MED_df[want_pred].copy()

percentage = 0.80
l = len(VA_MED_df[want_pred].iloc[:].to_list())
perci = int(l*percentage)

X_val = VA_MED_df[features].copy()
X_val_TR = VA_MED_df[features].copy().iloc[:perci,:]
X_val_VA = VA_MED_df[features].copy().iloc[perci:,:]

y_val = VA_MED_df[want_pred].copy()
y_val_TR = VA_MED_df[want_pred].iloc[:perci]
y_val_VA = VA_MED_df[want_pred].iloc[perci:]


# Local model
model = linear_model.LinearRegression()


# General
model.fit(X,y)
score = model.score(X,y)
pred = model.predict(VA_MED_df[features])
print('Model score:', score)
print("Validation MSE:", 
      mean_squared_error(VA_MED_df[want_pred], pred), '\n'*2)

# BASELINES

# Baseline 1 - msl = 15, st = -2, targ = 15
pred = list(map(CountWordsAudio_apply, VA_MED_df['aud_path']))
for i in range(len(pred)):
    pred[i] = pred[i] / VA_MED_df['num_words'][i]
print('Validation MSE of baseline "msl=15, st=-2, targ=15":', 
      mean_squared_error(y_val, pred), '\n')

# Baseline 2 - Average
pred = [st.mean(y_val)]*l
print('Validation MSE of baseline "Average":', 
      mean_squared_error(y_val, pred), '\n')

# Baseline 3 - Trend
mlr = LinearRegression()
mlr.fit(X_val_TR, y_val_TR)
pred = mlr.predict(X_val_VA)
score = mlr.score(X_val_VA, y_val_VA)
print('Model score of baseline "Trend":', score)
print('Validation MSE of baseline: "Trend"', 
      mean_squared_error(y_val_VA, pred), '\n')

Model score: 0.10092409179250594
Validation MSE: 328.8271170178382 


Validation MSE of baseline "msl=15, st=-2, targ=15": 1512.8667230214417 

Validation MSE of baseline "Average": 374.549375 

Model score of baseline "Trend": -0.057744034161325564
Validation MSE of baseline: "Trend" 280.1922628367017 



### b. 4D

In [6]:
features = quants.copy()
X = TR_MED_df[features].copy()
y = TR_MED_df[want_pred].copy()

percentage = 0.80
l = len(VA_MED_df[want_pred].iloc[:].to_list())
perci = int(l*percentage)

X_val = VA_MED_df[features].copy()
X_val_TR = VA_MED_df[features].copy().iloc[:perci,:]
X_val_VA = VA_MED_df[features].copy().iloc[perci:,:]

y_val = VA_MED_df[want_pred].copy()
y_val_TR = VA_MED_df[want_pred].iloc[:perci]
y_val_VA = VA_MED_df[want_pred].iloc[perci:]


# Local model
model = linear_model.LinearRegression()


# General
model.fit(X,y)
score = model.score(X,y)
pred = model.predict(VA_MED_df[features])
print('Model score:', score)
print("Validation MSE:", 
      mean_squared_error(VA_MED_df[want_pred], pred), '\n'*2)

# BASELINES

# Baseline 1 - msl = 15, st = -2, targ = 15
pred = list(map(CountWordsAudio_apply, VA_MED_df['aud_path']))
for i in range(len(pred)):
    pred[i] = pred[i] / VA_MED_df['num_words'][i]
print('Validation MSE of baseline "msl=15, st=-2, targ=15":', 
      mean_squared_error(y_val, pred), '\n')

# Baseline 2 - Average
pred = [st.mean(y_val)]*l
print('Validation MSE of baseline "Average":', 
      mean_squared_error(y_val, pred), '\n')

# Baseline 3 - Trend
mlr = LinearRegression()
mlr.fit(X_val_TR, y_val_TR)
pred = mlr.predict(X_val_VA)
score = mlr.score(X_val_VA, y_val_VA)
print('Model score of baseline "Trend":', score)
print('Validation MSE of baseline: "Trend"', 
      mean_squared_error(y_val_VA, pred), '\n')

Model score: 0.10714123172971535
Validation MSE: 326.9903387148712 


Validation MSE of baseline "msl=15, st=-2, targ=15": 1512.8667230214417 

Validation MSE of baseline "Average": 374.549375 

Model score of baseline "Trend": -0.08409017792176687
Validation MSE of baseline: "Trend" 287.17125340421853 



### 3. Multiclass Logistic Regression

*https://www.kaggle.com/code/satishgunjal/multiclass-logistic-regression-using-sklearn/notebook*

In [7]:
TR_MED_df = pd.read_csv(TTV[0][0:2].upper() + '_MED_df' + data_ext, sep=data_sep)
TE_MED_df = pd.read_csv(TTV[1][0:2].upper() + '_MED_df' + data_ext, sep=data_sep)
VA_MED_df = pd.read_csv(TTV[2][0:2].upper() + '_MED_df' + data_ext, sep=data_sep)

features = ['age_num', 'gender', 'accent', 
            'num_onsets', 'onset_stren_mean', 'dom_freq']
feats = features.copy()
feat = ['num_onsets']
two_feat = ['num_onsets', 'onset_stren_mean']
cats = ['age_num', 'gender', 'accent']
quants = ['num_onsets', 'onset_stren_mean', 'dom_freq']
want_pred = 'MEDIAN_func_num'




features = quants.copy()
X = TR_MED_df[features].copy()
y = TR_MED_df[want_pred].copy()

percentage = 0.80
l = len(VA_MED_df[want_pred].iloc[:].to_list())
perci = int(l*percentage)

X_val = VA_MED_df[features].copy()
X_val_TR = VA_MED_df[features].copy().iloc[:perci,:]
X_val_VA = VA_MED_df[features].copy().iloc[perci:,:]

y_val = VA_MED_df[want_pred].copy()
y_val_TR = VA_MED_df[want_pred].iloc[:perci]
y_val_VA = VA_MED_df[want_pred].iloc[perci:]

VA_MED_DF = VA_MED_df[features]

# Local model
dummies = pd.get_dummies(TR_MED_df[cats])
for i in range(3):
    X[feats[i]] = dummies.iloc[:,i]
    X_val[feats[i]] = dummies.iloc[:,i]
    X_val_TR[feats[i]] = dummies.iloc[:,i]
    X_val_VA[feats[i]] = dummies.iloc[:,i]
    VA_MED_DF[feats[i]] = dummies.iloc[:,i]
    features.append(feats[i])

model = linear_model.LogisticRegression(multi_class='ovr', solver='liblinear')

temp_VA_MED_df = VA_MED_df.copy()
VA_MED_df = VA_MED_DF.copy()
VA_MED_df[want_pred] = temp_VA_MED_df[want_pred]
VA_MED_df['aud_path'] = temp_VA_MED_df['aud_path']
VA_MED_df['num_words'] = temp_VA_MED_df['num_words']

# General
model.fit(X,y)
score = model.score(X,y)
pred = model.predict(VA_MED_df[features])
print('Model score:', score)
print("Validation MSE:", 
      mean_squared_error(VA_MED_df[want_pred], pred), '\n'*2)

# BASELINES

# Baseline 1 - msl = 15, st = -2, targ = 15
pred = list(map(CountWordsAudio_apply, VA_MED_df['aud_path']))
for i in range(len(pred)):
    pred[i] = pred[i] / VA_MED_df['num_words'][i]
print('Validation MSE of baseline "msl=15, st=-2, targ=15":', 
      mean_squared_error(y_val, pred), '\n')

# Baseline 2 - Average
pred = [st.mean(y_val)]*l
print('Validation MSE of baseline "Average":', 
      mean_squared_error(y_val, pred), '\n')

# Baseline 3 - Trend
mlr = LinearRegression()
mlr.fit(X_val_TR, y_val_TR)
pred = mlr.predict(X_val_VA)
score = mlr.score(X_val_VA, y_val_VA)
print('Model score of baseline "Trend":', score)
print('Validation MSE of baseline: "Trend"', 
      mean_squared_error(y_val_VA, pred), '\n')

VA_MED_df = temp_VA_MED_df.copy()

C:\Users\kitsu\AppData\Local\Temp/ipykernel_8636/2764156957.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  VA_MED_DF[feats[i]] = dummies.iloc[:,i]


Model score: 0.066
Validation MSE: 655.5825 


Validation MSE of baseline "msl=15, st=-2, targ=15": 1512.8667230214417 

Validation MSE of baseline "Average": 374.549375 

Model score of baseline "Trend": -0.09740236581278894
Validation MSE of baseline: "Trend" 290.69759997581633 



# Records

Model score: 0.08258976285484898
<br>
Validation MSE: 336.16257390230044 
<br>
<br>
Validation MSE of baseline "msl=15, st=-2, targ=15": 1512.8667230214417 
<br>
Validation MSE of baseline "Average": 374.549375 
<br>
Model score of baseline "Trend": -0.06770883999111232
<br>
Validation MSE of baseline: "Trend" 282.83190097598947 




> Model score: 0.10092409179250594
<br>
Validation MSE: 328.8271170178382 
<br>
<br>
Validation MSE of baseline "msl=15, st=-2, targ=15": 1512.8667230214417 
<br>
Validation MSE of baseline "Average": 374.549375 
<br>
Model score of baseline "Trend": -0.057744034161325564
<br>
Validation MSE of baseline: "Trend" 280.1922628367017




>> Model score: 0.10714123172971535
<br>
Validation MSE: 326.9903387148712 
<br>
<br>
Validation MSE of baseline "msl=15, st=-2, targ=15": 1512.8667230214417 
<br>
Validation MSE of baseline "Average": 374.549375 
<br>
Model score of baseline "Trend": -0.08409017792176687
<br>
Validation MSE of baseline: "Trend" 287.17125340421853 




>>> Model score: 0.066
<br>
Validation MSE: 655.5825 
<br>
<br>
Validation MSE of baseline "msl=15, st=-2, targ=15": 1512.8667230214417 
<br>
Validation MSE of baseline "Average": 374.549375 
<br>
Model score of baseline "Trend": -0.09740236581278894
<br>
Validation MSE of baseline: "Trend" 290.69759997581633 

# Multiple Linear Regression on ('num_onsets', 'onset_stren_mean', 'dom_freq')
## with close runner-up Multi Linear Regression on ('num_onsets', 'onset_stren_mean')

The latter of which we can graph!

# Testing (MLR)

In [8]:
TR_MED_df = pd.read_csv(TTV[0][0:2].upper() + '_MED_df' + data_ext, sep=data_sep)
TE_MED_df = pd.read_csv(TTV[1][0:2].upper() + '_MED_df' + data_ext, sep=data_sep)
VA_MED_df = pd.read_csv(TTV[2][0:2].upper() + '_MED_df' + data_ext, sep=data_sep)

features = ['age_num', 'gender', 'accent', 
            'num_onsets', 'onset_stren_mean', 'dom_freq']
feats = features.copy()
feat = ['num_onsets']
two_feat = ['num_onsets', 'onset_stren_mean']
cats = ['age_num', 'gender', 'accent']
quants = ['num_onsets', 'onset_stren_mean', 'dom_freq']
want_pred = 'MEDIAN_func_num'

### CAUTION ###
VA_MED_df = TE_MED_df.copy()

In [9]:
features = two_feat.copy()
X = TR_MED_df[features].copy()
y = TR_MED_df[want_pred].copy()

percentage = 0.80
l = len(VA_MED_df[want_pred].iloc[:].to_list())
perci = int(l*percentage)

X_val = VA_MED_df[features].copy()
X_val_TR = VA_MED_df[features].copy().iloc[:perci,:]
X_val_VA = VA_MED_df[features].copy().iloc[perci:,:]

y_val = VA_MED_df[want_pred].copy()
y_val_TR = VA_MED_df[want_pred].iloc[:perci]
y_val_VA = VA_MED_df[want_pred].iloc[perci:]


# Local model
model = linear_model.LinearRegression()


# General
model.fit(X,y)
score = model.score(X,y)
pred = model.predict(VA_MED_df[features])
print('Model score:', score)
print("Test MSE:", 
      mean_squared_error(VA_MED_df[want_pred], pred), '\n'*2)

# BASELINES

# Baseline 1 - msl = 15, st = -2, targ = 15
pred = list(map(CountWordsAudio_apply, VA_MED_df['aud_path']))
for i in range(len(pred)):
    pred[i] = pred[i] / VA_MED_df['num_words'][i]
print('Test MSE of baseline "msl=15, st=-2, targ=15":', 
      mean_squared_error(y_val, pred), '\n')

# Baseline 2 - Average
pred = [st.mean(y_val)]*l
print('Test MSE of baseline "Average":', 
      mean_squared_error(y_val, pred), '\n')

# Baseline 3 - Trend
mlr = LinearRegression()
mlr.fit(X_val_TR, y_val_TR)
pred = mlr.predict(X_val_VA)
score = mlr.score(X_val_VA, y_val_VA)
print('Model score of baseline "Trend":', score)
print('Test MSE of baseline: "Trend"', 
      mean_squared_error(y_val_VA, pred), '\n')

Model score: 0.10092409179250594
Test MSE: 346.6569226730464 


Test MSE of baseline "msl=15, st=-2, targ=15": 1464.71504459486 

Test MSE of baseline "Average": 357.134975 

Model score of baseline "Trend": -0.033649344640577894
Test MSE of baseline: "Trend" 320.3531271068908 



In [10]:
features = quants.copy()
X = TR_MED_df[features].copy()
y = TR_MED_df[want_pred].copy()

percentage = 0.80
l = len(VA_MED_df[want_pred].iloc[:].to_list())
perci = int(l*percentage)

X_val = VA_MED_df[features].copy()
X_val_TR = VA_MED_df[features].copy().iloc[:perci,:]
X_val_VA = VA_MED_df[features].copy().iloc[perci:,:]

y_val = VA_MED_df[want_pred].copy()
y_val_TR = VA_MED_df[want_pred].iloc[:perci]
y_val_VA = VA_MED_df[want_pred].iloc[perci:]


# Local model
model = linear_model.LinearRegression()


# General
model.fit(X,y)
score = model.score(X,y)
pred = model.predict(VA_MED_df[features])
print('Model score:', score)
print("Test MSE:", 
      mean_squared_error(VA_MED_df[want_pred], pred), '\n'*2)

# BASELINES

# Baseline 1 - msl = 15, st = -2, targ = 15
pred = list(map(CountWordsAudio_apply, VA_MED_df['aud_path']))
for i in range(len(pred)):
    pred[i] = pred[i] / VA_MED_df['num_words'][i]
print('Test MSE of baseline "msl=15, st=-2, targ=15":', 
      mean_squared_error(y_val, pred), '\n')

# Baseline 2 - Average
pred = [st.mean(y_val)]*l
print('Test MSE of baseline "Average":', 
      mean_squared_error(y_val, pred), '\n')

# Baseline 3 - Trend
mlr = LinearRegression()
mlr.fit(X_val_TR, y_val_TR)
pred = mlr.predict(X_val_VA)
score = mlr.score(X_val_VA, y_val_VA)
print('Model score of baseline "Trend":', score)
print('Test MSE of baseline: "Trend"', 
      mean_squared_error(y_val_VA, pred), '\n')

Model score: 0.10714123172971535
Test MSE: 338.12919801520286 


Test MSE of baseline "msl=15, st=-2, targ=15": 1464.71504459486 

Test MSE of baseline "Average": 357.134975 

Model score of baseline "Trend": -0.07232525832781445
Test MSE of baseline: "Trend" 332.3397354839615 



# Multiple Linear Regression WINS!

___

## Scratch work
